In [27]:
#importing all required modules
import pandas as pd
from sqlalchemy import create_engine, URL
import os
from dotenv import load_dotenv

In [70]:
#creating the connection to MySQL
#os.getenv("DATABASE_NAME")
load_dotenv()
url_object = URL.create(
    drivername = "mysql+pymysql",
    username = os.getenv("USER_NAME"),
    password = os.getenv("HOST_PASSWORD"),
    host = os.getenv("HOST"),
    port = os.getenv("PORT"),
    database = "british_airways_reviews" 
)
engine = create_engine(url_object)

In [30]:
#importing the table that wil soon be exported to SQL for some analysis
reviews = pd.read_excel("data\\SQL ba_reviews.xlsx", "ba_reviews_clean")
reviews.drop(columns=["Departed (City)", "Arrived"], axis = 1, inplace=True)
reviews.rename(columns={"date": "date_posted"}, inplace=True)
reviews.drop_duplicates()
reviews.head(5)

,header,author,date_posted,departed,traveller_type,seat_type,date_flown,recommended,trip_verified,rating,seat_comfort,cabin_staff_service,food_beverages,ground_service,value_for_money,entertainment
0,service was mediocre at best,Gary Storer,2023-10-03,United Kingdom,Couple Leisure,Economy Class,2023-01-10,no,Not Verified,2,2,3,1,2,2,-1
1,BA standards continue to decline,A Jensen,2023-10-02,United Kingdom,Business,Business Class,2023-01-09,no,Verified,2,2,1,2,1,1,-1
2,"won the race to the bottom""",John Rockett,2023-10-02,United Kingdom,Couple Leisure,Business Class,2023-01-09,no,Not Verified,2,2,3,2,1,1,-1
3,Not a reliable airline,Tatiana Bobrovskaya,2023-10-02,United Kingdom,Business,Economy Class,2023-01-10,no,Verified,3,4,4,2,1,1,-1
4,Very disappointed,Tom Slowbe,2023-09-28,United States,Couple Leisure,First Class,2023-01-09,no,Verified,1,1,4,4,3,2,3


In [32]:
countries = pd.read_csv("data\\Countries.csv").drop_duplicates()
countries.head(5)

,country,code,continent,region
0,Afghanistan,AFG,Asia,Southern Asia
1,Åland Islands,ALA,Europe,Northern Europe
2,Albania,ALB,Europe,Southern Europe
3,Algeria,DZA,Africa,Northern Africa
4,American Samoa,ASM,Oceania,Polynesia


In [4]:
#printing the dimensions of the table
print(f"rows:{reviews.shape[0]}, columns:{reviews.shape[1]}")
#checking the data type and number of non-null values in each column
reviews.info(verbose=True)

rows:1324, columns:16
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   header               1324 non-null   object        
 1   author               1324 non-null   object        
 2   date                 1324 non-null   datetime64[ns]
 3   Departed             1324 non-null   object        
 4   traveller_type       1323 non-null   object        
 5   seat_type            1324 non-null   object        
 6   date_flown           1324 non-null   datetime64[ns]
 7   recommended          1324 non-null   object        
 8   trip_verified        1324 non-null   object        
 9   rating               1324 non-null   int64         
 10  seat_comfort         1324 non-null   int64         
 11  cabin_staff_service  1324 non-null   int64         
 12  food_beverages       1324 non-null   int64         
 13  ground_serv

Searching for Viable Primary Keys

In [20]:
def is_candidate_key(pos_cand_columns: list, df: pd.DataFrame):
    """
    returns true if all the columns in pos_cand_columns unique identify every row and returns false 
    otherwise
    pos_cand_columns: A list of columns (strings)
    """

    #removes any rows with duplicate values in the columns in pos_cand_columns
    reduced = df.drop_duplicates(pos_cand_columns)
    # if the number of rows remain the same then pos_cand_columns is a candidate key
    #only works if there are no duplicates before this function is called
    if (reduced.shape[0] == df.shape[0]):
        return True
    return False


In [12]:
def print_result(col_subset: list, df: pd.DataFrame):
    #prints the results of whether col_subset is a candidate key or not
    if (is_candidate_key(col_subset, df)): 
        print(f"{col_subset} is a candidate key")
    else:
        print(f"{col_subset} is not a candidate key")
        

In [27]:
#checking the number of unique values per column
reviews.nunique()

header                 1300
author                  833
date                    938
Departed                 56
traveller_type            4
seat_type                 4
date_flown               88
recommended               2
trip_verified             2
rating                   10
seat_comfort              6
cabin_staff_service       6
food_beverages            6
ground_service            6
value_for_money           5
entertainment             6
dtype: int64

In [24]:
#Checking if (date_posted, author) is a candidate key for the table
print_result(["date_posted", "author"], reviews)
#(Date, Author) is NOT a valid Primary Key
print_result(["date_posted", "author", "departed"], reviews)

['date', 'author'] is not a candidate key
['date', 'author', 'departed'] is not a candidate key


In [45]:
#checking to see if all countries in reviews.departed are in countries.country
reviews_country_join = pd.merge(reviews, countries, how = "left", left_on="departed", right_on="country")
#display the number of countries that are in reviews.departed but not in countries.country
num_missing = reviews_country_join[reviews_country_join["country"].isna()]["country"].count()
print(f"number of countries with missing values in the join is {num_missing}")

number of countries with missing values in the join is 0


In [78]:
#exporting the Countried df to the SQL table
countries.to_sql(name = "Countries", con = engine, if_exists="append", index=False)
#exporting Reviews df to SQL table
reviews.to_sql(name= "BA_Reviews", con = engine, if_exists="append", index=False)
#testing to see if data was properly exported
pd.read_sql_query("SELECT country, COUNT(*) FROM Countries GROUP BY country", con=engine)

C:\Users\danie\AppData\Local\Temp\ipykernel_9692\1035140090.py:4: UserWarning: The provided table name 'BA_Reviews' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  reviews.to_sql(name= "BA_Reviews", con = engine, if_exists="append", index=False)


,country,COUNT(*)
0,Afghanistan,1
1,Åland Islands,1
2,Albania,1
3,Algeria,1
4,American Samoa,1
...,...,...
246,Wallis and Futuna Islands,1
247,Western Sahara,1
248,Yemen,1
249,Zambia,1


In [75]:
reviews.drop(columns=["reviewID"], inplace=True)